In [4]:
from tabula import read_pdf
df_temp = read_pdf('Análisis_Razonado92434000_202212 (1).pdf',pages='all')

c:\Users\ataglem\AppData\Local\anaconda3\Lib\site-packages\tabula\io.py:1057: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[c] = pd.to_numeric(df[c], errors="ignore")
c:\Users\ataglem\AppData\Local\anaconda3\Lib\site-packages\tabula\io.py:1057: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[c] = pd.to_numeric(df[c], errors="ignore")
c:\Users\ataglem\AppData\Local\anaconda3\Lib\site-packages\tabula\io.py:1057: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[c] = pd.to_numeric(df[c], errors="ignore")


In [193]:
import pandas as pd 
import numpy as np 


In [241]:
df=pd.read_csv("510000.csv")
df.set_index([df.columns.values[0],df.columns.values[1]],inplace=True)
df.columns.values


array(['Desde 2018-01-01 Hasta 2018-03-31',
       'Desde 2017-01-01 Hasta 2017-03-31',
       'Desde 2018-01-01 Hasta 2018-06-30',
       'Desde 2017-01-01 Hasta 2017-06-30',
       'Desde 2018-01-01 Hasta 2018-09-30',
       'Desde 2017-01-01 Hasta 2017-09-30',
       'Desde 2018-01-01 Hasta 2018-12-31',
       'Desde 2017-01-01 Hasta 2017-12-31',
       'Desde 2019-01-01 Hasta 2019-03-31',
       'Desde 2019-01-01 Hasta 2019-06-30',
       'Desde 2019-01-01 Hasta 2019-09-30',
       'Desde 2019-01-01 Hasta 2019-12-31',
       'Desde 2020-01-01 Hasta 2020-03-31',
       'Desde 2020-01-01 Hasta 2020-06-30',
       'Desde 2020-01-01 Hasta 2020-09-30',
       'Desde 2020-01-01 Hasta 2020-12-31'], dtype=object)

In [239]:
cadena = 'Desde 2018-01-01 Hasta 2018-03-31'

# Número que buscas
numero_buscado = "01"

# Posición en la que debe estar el número
posicion_deseada = 11

# Verifica si el número está en la posición especificada en la cadena
tiene_numero_en_posicion(cadena, numero_buscado, posicion_deseada) and tiene_numero_en_posicion(cadena, "03", 28)


True

In [ ]:
def column_is_quarter(name,quarter): # see if quarter and change name 


def change_cols_name(df): #to cols



    


In [213]:
def tiene_numero_en_posicion(cadena, numero, posicion):
    # Construye el patrón para buscar el número en la posición específica
    patron = r'^.{%d}%s' % (posicion , numero)
    # Busca la coincidencia
    coincidencia = re.search(patron, cadena)
    # Devuelve True si se encuentra la coincidencia, False en caso contrario
    return bool(coincidencia)

In [208]:
import re
lista=list(df.columns.values)


pattern=r'^.{%d}%s'% (posicion - 1, numero)

In [ ]:
matched_strings = [s for s in lista if re.match(pattern, s)]


['Desde 2018-01-01 Hasta 2018-03-31',
 'Desde 2017-01-01 Hasta 2017-03-31',
 'Desde 2018-01-01 Hasta 2018-06-30',
 'Desde 2018-04-01 Hasta 2018-06-30',
 'Desde 2017-01-01 Hasta 2017-06-30',
 'Desde 2017-04-01 Hasta 2017-06-30',
 'Desde 2018-01-01 Hasta 2018-09-30',
 'Desde 2018-07-01 Hasta 2018-09-30',
 'Desde 2017-01-01 Hasta 2017-09-30',
 'Desde 2017-07-01 Hasta 2017-09-30',
 'Desde 2018-01-01 Hasta 2018-12-31',
 'Desde 2017-01-01 Hasta 2017-12-31',
 'Desde 2019-01-01 Hasta 2019-03-31',
 'Desde 2019-01-01 Hasta 2019-06-30',
 'Desde 2019-04-01 Hasta 2019-06-30',
 'Desde 2019-01-01 Hasta 2019-09-30',
 'Desde 2019-07-01 Hasta 2019-09-30',
 'Desde 2019-01-01 Hasta 2019-12-31',
 'Desde 2020-01-01 Hasta 2020-03-31',
 'Desde 2020-01-01 Hasta 2020-06-30',
 'Desde 2020-04-01 Hasta 2020-06-30',
 'Desde 2020-01-01 Hasta 2020-09-30',
 'Desde 2020-07-01 Hasta 2020-09-30',
 'Desde 2020-01-01 Hasta 2020-12-31']

In [195]:
df.set_index([df.columns.values[i] for i in [0,1]],inplace=True)